In [1]:
import silence_tensorflow.auto
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
from tensorflow_probability import bijectors as tfb
from numpy.random import gamma

from tfprojgamma import ProjectedGamma
# Set random seeds for reproducibility
np.random.seed(1)
tf.random.set_seed(1)

In [2]:
# Set up shape parameters
alpha_true = gamma(size = 5, shape = 1.5)
print(alpha_true)
# dist1 = ProjectedGamma(alpha_true, 10)
# Yp    = dist1.sample(200)
dist1 = tfd.Dirichlet(concentration = alpha_true)
Yp = dist1.sample(200)

[3.94761907 0.62279234 0.68411215 0.34912511 4.2482095 ]


In [3]:
# prior shape parameters
log_alpha_0 = tf.zeros(5, dtype = tf.float64)
Sigma       = tf.eye(5, dtype = tf.float64)

In [4]:
# define generator
def generative_model(log_alpha_0, n_samples):
    log_alpha = yield tfd.JointDistributionCoroutine.Root(
        tfd.MultivariateNormalDiag(
            loc = log_alpha_0, scale = tf.ones(5, dtype = tf.float64), name = 'log_alpha'
            ),
        )
    Yp = yield tfd.Sample(
        # ProjectedGamma(concentration = tf.exp(log_alpha) * tf.ones((n_samples, 5), dtype = tf.float64)),
        tfd.Dirichlet(concentration = tf.exp(log_alpha) * tf.ones((n_samples, 5), dtype = tf.float64)),
        name = 'Yp',
        )
model_joint = tfd.JointDistributionCoroutineAutoBatched(
    lambda: generative_model(log_alpha_0, 200),
    )

In [5]:
model_joint_log_prob = lambda log_alpha: model_joint.log_prob(log_alpha, Yp)

In [7]:
precison_to_unconstrained = tfb.Chain([
    # step 3: flatten the lower triangular portion of the matrix
    tfb.Invert(tfb.FillTriangular(validate_args = True)),
    # step 2: take the log of the diagonals    
    tfb.TransformDiagonal(tfb.Invert(tfb.Exp(validate_args = True))),
    # step 1: decompose the precision matrix into its Cholesky factors
    tfb.Invert(tfb.CholeskyOuterProduct(validate_args = True)),
    ])

In [ ]:
# New Style: Make the variational Parameters

q_mu = tf.Variable(log_alpha_0, name = 'Mu Surrogate (mean of log alpha)')
q_E  = tf.Variable(precision_to_unconstrained.forward



In [ ]:
# Make the variational parameters
q_log_alpha = tf.Variable(log_alpha_0, name = 'log Alpha (surrogate)')
q_Sigma     = tf.Variable(Sigma, name = 'Sigma (surrogate)') # assume this is precision parameterized...

In [ ]:
surrogate_posterior = tfd.MultivariateNormalFullCovariance(loc = q_log_alpha, covariance_matrix = q_Sigma)

In [ ]:
with tf.GradientTape() as g:
    samples = surrogate_posterior.sample(50)
    neg_elbo = - tf.reduce_mean(model_joint_log_prob(samples) - surrogate_posterior.log_prob(samples))

In [ ]:
path = tfp.vi.fit_surrogate_posterior(
    target_log_prob_fn = model_joint_log_prob,
    surrogate_posterior = surrogate_posterior,
    optimizer = tf.optimizers.Adam(.2),
    num_steps = 1000,
    sample_size = 500,
    )

In [ ]:
g.gradient(neg_elbo, surrogate_posterior.trainable_variables)